Importing all required packages

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
IMAGE_SIZE = [224, 224,3]

Setting training and Testing path

In [5]:
train_path = '/content/drive/My Drive/transfer learning masks/covid masks/train'
valid_path = '/content/drive/My Drive/transfer learning masks/covid masks/test'
bs=16

Preprocessing the Images

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range = 20,
                    zoom_range = 0.15,
                    width_shift_range = 0.2,
                    height_shift_range = 0.2,
                    shear_range = 0.15,
                    horizontal_flip = True,
                    fill_mode="nearest")

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = bs,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = bs,
                                            class_mode = 'categorical')

Found 919 images belonging to 2 classes.
Found 397 images belonging to 2 classes.


Using ResNet50 for Transfer Learning on our dataset

In [7]:
Res = ResNet50(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
last_layer=Res.output            
last_layer = tf.keras.layers.AveragePooling2D(pool_size = (7,7))(last_layer)
last_layer = tf.keras.layers.Flatten()(last_layer)
last_layer = tf.keras.layers.Dense(256, activation = 'relu')(last_layer)
last_layer = tf.keras.layers.Dropout(0.5)(last_layer)
last_layer = tf.keras.layers.Dense(2, activation = 'softmax')(last_layer)

Accessing masked and unmasked images files from our folder

In [8]:
from glob import glob
folders = glob('/content/drive/My Drive/transfer learning masks/covid masks/train/*')

Creating our model

In [9]:
from keras.models import Model
model = Model(inputs=Res.input, outputs=last_layer)

Checking summary of our model for checking the layers in the network

In [10]:
model.summary()


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

Compiling model with hyperparameters

In [11]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Fitting the model with our dataset

In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=26,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/26
58/58 [==============================] - 23s 392ms/step - loss: 0.4260 - accuracy: 0.8890 - val_loss: 53194.9180 - val_accuracy: 0.5139
Epoch 2/26
58/58 [==============================] - 21s 368ms/step - loss: 0.2104 - accuracy: 0.9358 - val_loss: 169.2641 - val_accuracy: 0.5139
Epoch 3/26
58/58 [==============================] - 21s 369ms/step - loss: 0.1487 - accuracy: 0.9521 - val_loss: 0.9671 - val_accuracy: 0.5139
Epoch 4/26
58/58 [==============================] - 21s 365ms/step - loss: 0.0944 - accuracy: 0.9674 - val_loss: 0.9088 - val_accuracy: 0.5139
Epoch 5/26
58/58 [==============================] - 21s 368ms/step - loss: 0.1154 - accuracy: 0.9706 - val_loss: 0.9577 - val_accuracy: 0.5139
Epoch 6/26
58/58 [==============================] - 21s 366ms/step - loss: 0.1378 - accuracy: 0.9521 - val_loss: 0.8590 - val_accuracy: 0.5139
Epoch 7/26
58/58 [==============================] - 21s 364m

Evaluating the model

In [ ]:
 train_acc=model.evaluate(training_set,verbose=2)
 test_acc = model.evaluate(test_set, verbose=2)

In [ ]:
train_acc

In [ ]:
test_acc